#### Libraries

In [44]:
import geopandas as gpd
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

#### Importing Data

In [24]:
#Ecoregion data from EPA
ecoregions = gpd.read_file("us_eco_l3.shp")

In [11]:
#MTBS Wildfire data
mtbs_fires = gpd.read_file("mtbs_fod_pts_DD.shp")

In [8]:
#Climate Monitor Locations
metadata_col_specs = [(0,12),(12,21),(21,31),(31,38),(38,41),(41,72),(72,76),(76,80),(80,86)]

metadata_names = ["ID","LATITUDE","LONGITUDE","ELEVATION","STATE","NAME","GSN FLAG","HCN/CRN FLAG","WMO ID"]

metadata_dtype = {"ID": str, "LATITUDE": float, "LONGITUDE":float, "ELEVATION": float, "STATE": str,"NAME": str,
    "GSN FLAG": str,"HCN/CRN FLAG": str,"WMO ID": str}

def read_station_metadata(filename="ghcnd-stations.txt"):
    """Reads in station metadata

    :filename: ghcnd station metadata file.
    :returns: station metadata as a pandas Dataframe

    """
    df = pd.read_fwf(filename, metadata_col_specs, names=metadata_names, dtype=metadata_dtype)

    return df

monitors = read_station_metadata()

#### Checking/Cleaning Data

In [25]:
#Selecting columns from ecoregion data
eco_join = ecoregions[['NA_L3NAME', 'NA_L2NAME', 'NA_L1NAME', 'geometry']]

In [20]:
#Converting monitor dataframe to geodataframe
monitor_geo = gpd.GeoDataFrame(
    monitors, geometry=gpd.points_from_xy(monitors.LONGITUDE, monitors.LATITUDE))
monitor_geo = monitor_geo.set_crs('EPSG:4269')

In [26]:
#Checking to make sure Coordinate Reference Systems match
print(eco_join.crs == mtbs_fires.crs)
print(monitor_geo.crs == mtbs_fires.crs)

False
True


In [27]:
#Matching the CRS of both Ecoregions and Monitors to the MTBS database
eco_join = eco_join.to_crs(mtbs_fires.crs)
monitor_geo = monitor_geo.to_crs(mtbs_fires.crs)

In [28]:
print(eco_join.crs == mtbs_fires.crs)
print(monitor_geo.crs == mtbs_fires.crs)

True
True


#### Joining Data

In [29]:
#Join MTBS and Ecoregions
eco_mtbs = gpd.sjoin(mtbs_fires, eco_join, how='left', op='intersects')
eco_mtbs.drop(columns=['geometry', 'index_right'], inplace=True)

In [42]:
#Join Monitors and Ecoregions
eco_monitor = gpd.sjoin(monitor_geo, eco_join, how='left', op='intersects')

#### Sorting Joined Data

In [45]:
#US only weather monitors
eco_monitor_us = eco_monitor[eco_monitor['ID'].str.contains('US')]
eco_monitor_us.drop(columns=['NAME', 'GSN FLAG', 'HCN/CRN FLAG', 'WMO ID', 'geometry', 'index_right'],
                 inplace=True)
eco_monitor_us.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NA_L3NAME,NA_L2NAME,NA_L1NAME
52427,US009052008,43.7333,-96.6333,482.0,SD,Western Corn Belt Plains,TEMPERATE PRAIRIES,GREAT PLAINS
52428,US10RMHS145,40.5268,-105.1113,1569.1,CO,High Plains,SOUTH CENTRAL SEMI-ARID PRAIRIES,GREAT PLAINS
52429,US10adam001,40.5680,-98.5069,598.0,NE,Central Great Plains,SOUTH CENTRAL SEMI-ARID PRAIRIES,GREAT PLAINS
52430,US10adam002,40.5093,-98.5493,601.1,NE,Central Great Plains,SOUTH CENTRAL SEMI-ARID PRAIRIES,GREAT PLAINS
52431,US10adam003,40.4663,-98.6537,615.1,NE,Central Great Plains,SOUTH CENTRAL SEMI-ARID PRAIRIES,GREAT PLAINS


In [49]:
#creating a monitor database that have only monitors in ecoregions where fires have occured
sub_regions_mtbs = eco_mtbs['NA_L3NAME'].unique()
eco_monitor_mtbs = eco_monitor_us[eco_monitor_us['NA_L3NAME'].isin(sub_regions_mtbs)]

#### Writing Data to CSVs

In [51]:
eco_mtbs.to_csv('MTBS_and_Region.csv')

In [52]:
eco_monitor_mtbs.to_csv('Monitor_by_Region.csv')